In [1]:
import pandas as pd
import os
import numpy as np
from scipy.spatial.distance import pdist, squareform

abd_path = '../../ERP10700.bracken.mpa/ERR912196.bracken.report.mpa.txt'
idir = '.'
abd = pd.read_csv(abd_path, sep='\t', header=None, index_col=0)
level = 's'
take_sp_list = []
rename_dict = {}
for idx in abd.index:
    last_l = idx.split('|')[-1]
    if last_l[0] == level:
        take_sp_list.append(idx)
        rename_dict[idx] = last_l
abd = abd.loc[take_sp_list, ]
abd.rename(index=rename_dict, inplace=True)
abd = abd.div(abd.sum(axis=0))


In [2]:
def make_d(GCN_path, sep='\t'):
    ref_GCN= pd.read_csv(GCN_path, sep=sep, index_col=0, header=0)
    ref_GCN = ref_GCN.T
    sp_list = list(ref_GCN.columns)
    distance_compressed = pdist(ref_GCN.values.T, Jaccard_distance)
    distance_matrix = squareform(distance_compressed)
    distance_df = pd.DataFrame(distance_matrix, columns = sp_list, index = sp_list)
    return distance_df


def nfr(d_df, profile):
    sp_list = list(set(list(profile.index)).intersection(set(list(d_df.index))))
    n = len(sp_list)
    corr = np.ones(shape=(n, n)) - d_df.loc[sp_list, sp_list].values
    np.fill_diagonal(corr, 0)
    a = np.array(profile.loc[sp_list, 1])
    inter_matrix = np.dot(a.reshape(len(a), 1),a.reshape(1, len(a)))
    np.fill_diagonal(inter_matrix, 0)
    td = np.sum(inter_matrix)/2
    fr = np.sum(np.multiply(inter_matrix, corr))/2
    fr_df = pd.DataFrame(np.multiply(inter_matrix, corr), index=sp_list, columns=sp_list)
    profile = profile.loc[sp_list, 1]
    if td == 0:
        return 0
    return fr/td, fr_df, profile

In [3]:
GCN_path = '../GCN_s.tsv'

ref_GCN= pd.read_csv(GCN_path, sep='\t', index_col=0, header=0)
ref_GCN = ref_GCN.T
ref_GCN

FileNotFoundError: [Errno 2] No such file or directory: '../GCN_s.tsv'

In [4]:
def Jaccard_distance(x, y):
    union = sum(np.maximum(x, y))
    intersection = sum(np.minimum(x, y))
    # print(intersection, union)
    if union != 0:
        return 1 - intersection/union
    else:
        # print("union size is 0")
        return 1


In [13]:
sp_list = list(ref_GCN.columns)
distance_compressed = pdist(ref_GCN.values.T, Jaccard_distance)
distance_matrix = squareform(distance_compressed)
distance_df = pd.DataFrame(distance_matrix, columns = sp_list, index = sp_list)


In [7]:
GCN_d_path = 'sp_d.tsv'
'''
rename_dict = {}
for idx in distance_df.index:
    rename_dict[idx] = idx.replace(' ', '_')
distance_df.rename(index=rename_dict, columns=rename_dict, inplace=True)

d_df = distance_df.to_csv(GCN_d_path, sep='\t', index=True, header=True)
'''


"\nrename_dict = {}\nfor idx in distance_df.index:\n    rename_dict[idx] = idx.replace(' ', '_')\ndistance_df.rename(index=rename_dict, columns=rename_dict, inplace=True)\n\nd_df = distance_df.to_csv(GCN_d_path, sep='\t', index=True, header=True)\n"

In [8]:

d_df = pd.read_csv(GCN_d_path, sep='\t', index_col=0, header=0)

nfr, fr_df, profile = nfr(d_df, abd)


In [29]:
profile.to_csv(os.path.join(idir, 'abd.tsv'), sep='\t', header=False, index=True)
with open(os.path.join(idir, 'nfr.txt'), 'w') as f:
    f.write(str(nfr))
fr_df.to_csv(os.path.join(idir, 'net_df.tsv'), sep='\t', header=True, index=True)

In [34]:
edge_df = fr_df.stack().reset_index()


In [35]:
edge_df

,level_0,level_1,0
0,s__Collinsella_sp900768795,s__Collinsella_sp900768795,0.000000e+00
1,s__Collinsella_sp900768795,s__Eubacterium_R_sp900767025,1.232303e-12
2,s__Collinsella_sp900768795,s__Christensenella_timonensis,4.802477e-12
3,s__Collinsella_sp900768795,s__Alistipes_sp900021155,6.815499e-11
4,s__Collinsella_sp900768795,s__Akkermansia_sp001580195,5.170685e-13
...,...,...,...
8886356,s__Providencia_stuartii,s__Prevotella_sp000436595,1.533235e-11
8886357,s__Providencia_stuartii,s__Bilophila_wadsworthia,6.336325e-10
8886358,s__Providencia_stuartii,s__Eggerthella_lenta,1.983817e-12
8886359,s__Providencia_stuartii,s__Gemmiger_variabilis_B,6.468064e-12


In [11]:
row_index, col_index = np.tril_indices(len(fr_df), k=0)

# 将下半三角元素设置为 0
fr_df.values[row_index, col_index] = 0
edge_df = fr_df.stack().reset_index()
edge_df = edge_df[edge_df[0] != 0]


In [13]:
edge_df = edge_df[edge_df[0] != 0]

In [14]:
edge_df

,level_0,level_1,0
1,s__RUG420_sp900771085,s__Ruminococcus_D_sp000434695,6.285892e-13
2,s__RUG420_sp900771085,s__CAG-485_sp900766975,1.371256e-13
3,s__RUG420_sp900771085,s__UMGS1260_sp900550105,4.859765e-14
4,s__RUG420_sp900771085,s__Collinsella_sp900552875,1.514964e-12
5,s__RUG420_sp900771085,s__Collinsella_sp900556365,2.583513e-14
...,...,...,...
8877416,s__Dysosmobacter_sp900544105,s__Succiniclasticum_sp900544275,1.752448e-09
8877417,s__Dysosmobacter_sp900544105,s__Prevotella_timonensis,4.009662e-10
8880397,s__CAG-269_sp900554365,s__Succiniclasticum_sp900544275,1.575452e-11
8880398,s__CAG-269_sp900554365,s__Prevotella_timonensis,4.297347e-12


In [15]:
(fr_df.shape[0]**2 - fr_df.shape[0])/2

4441690.0